In [13]:
from keras.models import Sequential, Model

from keras.layers import Dense, Rescaling, Embedding, Reshape, Conv2D, MaxPooling2D, Flatten
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ModelCheckpoint
from IPython.display import SVG

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.data import AUTOTUNE


In [6]:
from tensorflow.keras.applications.vgg16 import VGG16
#This is from someone's git repo. They created a py package that will plot the loss function as your model trains
import livelossplot
plot_losses = livelossplot.PlotLossesKeras()

In [3]:
batch_size = 32
img_height = 256
img_width = 256

img_folder = 'C:/Users/louisa/Documents/GitHub/DeCoreIdea/decore_style'

In [4]:
#obtaining the training dataset
train_ds = image_dataset_from_directory(
  img_folder,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2420 files belonging to 8 classes.
Using 1936 files for training.


In [5]:
#obtaining the validation dataset
val_ds = image_dataset_from_directory(
  img_folder,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2420 files belonging to 8 classes.
Using 484 files for validation.


In [10]:
# Instantiate convolutional base from imagenet data
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

58900480/58889256 [==============================] - 2s 0us/step


In [18]:
flat1 = Flatten()(conv_base.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(8, activation='softmax')(class1)
# define new model
model = Model(inputs=conv_base.inputs, outputs=output)
# summarize
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [19]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(train_ds, epochs=5, verbose=1)  # track progress as we fit

Epoch 1/5


KeyboardInterrupt: 

In [ ]:
#trying to get a CNN to work for the 8 classes.
model = Sequential([
        Rescaling(1./255),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        Conv2D(32, 6, activation='relu'),
        MaxPooling2D(),
        Conv2D(32, 6, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(8)])

In [21]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6535045178786249138
]
